# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [16]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display, update_display

In [23]:
# constants

MODEL_GPT = 'openai/gpt-4o-mini'
MODEL_LLAMA = "meta-llama/llama-3.2-3b-instruct"

In [5]:
# set up environment
load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")
api_url = os.getenv("OPENROUTER_API_BASE")
openai = OpenAI(base_url=api_url, api_key=api_key)

In [8]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = {
    "role": "system",
    "content": "You are a helpful assistant that explains code."
}

user_prompt = {
    "role": "user",
    "content": question
}

In [27]:
def display_streamed_response(response, model_name):
    display(Markdown(f"**{model_name} response:**"))
    result = ""
    display_handle = display(Markdown(result), display_id=True)
    for chunk in response:
        content = chunk.choices[0].delta.content or ""
        result += content
        update_display(Markdown(result), display_id=display_handle.display_id)

In [17]:
# Get gpt-4o-mini to answer, with streaming
response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[system_prompt, user_prompt],
    stream=True
)

display(Markdown("**GPT-4o-mini response:**"))
result = ""
display_handle = display(Markdown(result), display_id=True)
for chunk in response:
    content = chunk.choices[0].delta.content or ""
    result += content
    update_display(Markdown(result), display_id=display_handle.display_id)

**GPT-4o-mini response:**

The given code snippet is using Python's `yield from` statement combined with a set comprehension. Let's break down the components to understand what it does and why.

### Components Breakdown

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This part of the code is a set comprehension that iterates over a collection (in this case, `books`).
   - For each `book` in the `books` iterable, it calls `book.get("author")` to attempt to retrieve the value associated with the key `"author"`.
   - The `if book.get("author")` condition filters out any books that do not have an author (i.e., it ensures that `None` or any falsy value for `book.get("author")` is excluded).
   - As a result, this comprehension creates a set containing all unique authors from the `books` collection, where an author exists.

2. **`yield from`**: 
   ```python
   yield from <iterable>
   ```
   - The `yield from` statement is used inside a generator function. It delegates the yielding of values to another iterable (in this case, the set produced by the comprehension).
   - It will yield each item from the set one at a time. If the generator this code is part of is called, it will produce values one by one, and each value will be deduplicated since it comes from a set.

### Overall Functionality

Putting it all together, this line of code is likely part of a generator function that returns the authors of the books in the `books` list, yielding each author exactly once (due to the nature of sets ensuring uniqueness). Specifically, it:

1. Iterates through each book in the `books` list.
2. Retrieves the author if it exists.
3. Constructs a set of authors to eliminate duplicates.
4. Yields each author one by one to the caller when this generator function is used.

### Example Usage

If you have a list of dictionaries representing books like this:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
    {"title": "Book E", "author": "Author 3"},
]
```

The code would yield:
- "Author 1"
- "Author 2"
- "Author 3"

Each author is yielded once, and the code neatly avoids yielding any empty or non-existent authors. 

### Conclusion

This combination of a set comprehension with `yield from` provides a concise way to produce a unique list of authors from a collection of book dictionaries while using the generator functionality for efficient iteration.

In [28]:
# Get Llama 3.2 to answer
response = openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[system_prompt, user_prompt],
    stream=True
)

display_streamed_response(response, "Llama 3.2")

**Llama 3.2 response:**

This code uses a combination of Python's `yield` keyword and the `from` keyword to create a generator expression that fetches author names from a list of book objects.

Here's a breakdown of what's happening:

- `book.get("author")`: This expression attempts to retrieve the value associated with the key `"author"` from a book object. If the key does not exist, it returns `None`.

- `{... for book in books if book.get("author")}`: This is a dictionary comprehension (a dictionary equivalent of a list comprehension) that filters the `books` list to include only those books that have an `"author"` key. The expression `book.get("author")` is evaluated for each book in the list, and only books that pass the condition `if book.get("author")` (i.e., those with an `"author"` key) are included.

- `yield from {...}`: This is where things get interesting. The `yield from` syntax is used to delegate to a subgenerator. It allows the outer generator to produce a sequence that is created by the subgenerator.

- `yield from {...}` is equivalent to writing `for item in subgenerator: yield item`, where `subgenerator` is a generator expression or a generator function.

So, when put together, this code is essentially saying:

- For each book in the `books` list that has an `"author"` key,
- Yield (produce) the author name (i.e., the value retrieved by `book.get("author")`).

In other words, this code is generating a sequence of author names, one for each book that has an author.